# Домашнее задание.
### Работа с BigQuery с помощью SQL запросов

Сначала подготовим среду для работы с BigQuery, далее к решению предлагаются задачи

In [ ]:
import os

# тут должен быть путь к вашему ключу
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= \
"/home/emil/Documents/steel-totality-286807-6316434751b0.json"

In [ ]:
# Для уставновки пакетов, используйте следующующие команды
!conda install -c conda-forge google-cloud-bigquery -y
!conda install -c conda-forge pandas -y
!pip install pyarrow

# Если в ходе решения вам понадобится установка дополнительных библиотек, 
# устанавливайте их с помощью pip или conda
# Не стесняйтесь копировать текст ошибок прямиком в поисковик!

Для части задач будем использовать датасет `chicago_crime`

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

### Задача 1

Напишите код, необходимый для подсчета числа таблиц в датасете Chicago Crime

In [ ]:
# Ваш код

### Задача 2

Как много колонок в таблице `crime` имеют тип `TIMESTAMP`?

In [ ]:
# Ваш код

### Задача 3.

Для того, чтобы была возможность отобразить преступления на карте, нужно создать массив из пар координат. Получите эти данные из таблицы и сохраните их в списке

In [ ]:
# fields_for_plotting = [(x1, y1), ...]
# Ваш код

Далее работаем с датасетом `openaq` с таблицей `global_air_quality`

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

dataset_ref = client.dataset("openaq", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("global_air_quality")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

### Задача 4

Напишите запрос к таблице, в результате которого мы узнаем, какие страны исползуют единицы `ppm` для измерения загрязненности воздуха.

Напишите свой запрос в ячейке ниже, и поместите в переменную `first_query`.

Напомню, как выглядят запросы такого типа:
```SQL
query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
        """
```

In [ ]:
first_query = # введите ваш запрос

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)

first_results = first_query_job.to_dataframe()

print(first_results.head())

### Задача 5

В каких местах уровень загрязнения воздуха равен нулю?
1. Определите запрос `zer_pollution_query`, выбирающий все колонки таблицы, где `value` равен нулю
2. В переменнную `zero_pollution_results` сохраните `pd.DataFrame` этой отфильтрованной таблицы

In [ ]:
zero_pollution_query = # ваш код запроса

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

zero_pollution_results = # преобразование результатов к DataFrame

print(zero_pollution_results.head())

Теперь рассмотрим таблицу комментариев из датасета новостного портала `hacker_news`

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("comments")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

### Задача 6

Хакер-ньюс разыгрывают премию между авторами, у которых более 10 тысяч новостей. Давайте, с помощью SQL запроса, найдем всех авторов с более 10_000 постов и количество их постов. Назовите колонку с количеством постов `NumPosts`

Здесь нам пригодится оператор `GROUP BY ... HAVING`, пример запроса:

```SQL
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.comments`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """
        ```

In [ ]:
prolific_commenters_query = # ваш запрос

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

prolific_commenters = query_job.to_dataframe()

print(prolific_commenters.head())

### Задача 7. 

Как много комментариев в таблице удалено? Используйте поле `deleted` со значением `True`, чтобы вывести суммарное число удаленных комментариев

In [ ]:
deleted_posts_query = # ваш запрос

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(deleted_posts_query, job_config=safe_config)

deleted_posts = query_job.to_dataframe()

print(prolific_commenters.head())

num_deleted_posts = # сохраните это число в переменную

Следующий датасет к рассмотрению --- `world_bank_intl_education` и таблица `international_education`

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("international_education")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

В этой таблице значения в колонке `indicator_code` обозначают, что за данные представлены в строке.

Например, индикатор `SE.XPD.TOTL.GD.ZS` отвечает за `Государственные расходы на образование в процентах от ВВП (%)`

### Задача 8

Напишите запрос, в результате которого будет видно, какие страны тратят наибольшую долю ВВП на образование?

Сначала нужно выделить все строки, где `SE.XPD.TOTL.GD.ZS`, и напишите запрос, который вернет среднее значение колонки `value` для каждой страны в датасете, в период 2010-2017 гг (обе границы включительно)

Требования:
* Результаты долдны иметь имена стран, а не коды стран. Для каждой страны одна строка в результате
* Используйте имя `avg_ed_spending_pct` для колонки, которую создадите с помощбю `AVG()`
* Упорядочьте результаты так, чтобы страны с наибольшим средним отображались первыми. 

Пример похожего запроса на другом датасете:

```SQL
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
        ```

In [ ]:
country_spend_pct_query = """
                          SELECT _____
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE ____
                          GROUP BY ____
                          ORDER BY ____
                          """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

country_spending_results = country_spend_pct_query_job.to_dataframe()

print(country_spending_results.head())

### Задача 9

В этой таблице тысячи различных кодов в поле `indicator`, чтобы ознакомится с ними всеми потребуется очень много времени. Множество кодов доступно для ограниченного числа стран, мы можем отбросить из рассмотрения коды, которые встречаются редко.

Напишите такой запрос, чтобы получить таблицу, содержащую коды и имена `indicator_name`, для тех `indicator`, которые встречаются более 175 раз в 2016м году.

Требования:

* Одна строка на одно имя и `indicator` код
* Колонки должны называться: `indicator_code`, `indicator_name` и `num_rows`
* Результат должен содержать только коды, где >= 175 строк
* Чтобы получить и `indicator_code`, и `indicator_name` в результирующем DataFrame, нужно добавить оба поля в `SELECT` (в дополнении к агрегации `COUNT()`). Это значит, что их оба нужно добавить и в `GROUP BY`
* Упорядочьте результаты в порядке убывания (сверху вниз)

In [ ]:
code_count_query = # ваш запрос

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

code_count_results = code_count_query_job.to_dataframe()

print(code_count_results.head())

Наконец, перейдем к `JOIN`. Работать будем, как вы могли догадаться, с новым датасетом. С таблицами `posts_questions` и `posts_questions` из датасета `stackoverflow`


In [ ]:
from google.cloud import bigquery

lient = bigquery.Client()

dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

table_ref = dataset_ref.table("posts_questions")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

Таблица ответов

In [ ]:
table_ref = dataset_ref.table("posts_answers")

table = client.get_table(table_ref)

client.list_rows(table, max_results=5).to_dataframe()

### Задача 10

Как долго приходится ждать, чтобы получить ответ на вопрос? 

Рассмотрим запрос, с его помощью получаем информацию по вопросам, заданным в 2018 году. Он возвращает:
* `q_id` идентификатор вопроса
* `time_to_answer` сколько секунд прошло до ответа на вопрос

Ячейку ниже нужно выполнить не изменяя

In [ ]:
first_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

first_result = client.query(first_query).result().to_dataframe()
print("Процент отвеченных вопросов: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Количество вопросов:", len(first_result))
first_result.head()

Видим подозрительные результаты. Очевидно, с запросом что-то не так.

* Исходя из запроса, 100% вопросов в 2018 получили ответы. Но статистика сайта говорит, что только 80% всех вопросов бывают с ответами
* Полное число вопросов удивительно мало 134 тысячи. 

Учитывая проблемы, указанные выше, кажется что, выбранный тип `JOIN` отфильтровал неотвеченные вопросы и сохранил лишь те, что с ответами. Предлагается изменить запрос так, чтобы достичь желаемого поведения, использую один из методов для джойна таблиц (`INNER, LEFT, RIGHT` или `FULL`)

In [ ]:
correct_query = # ваш запрос 

first_result = client.query(correct_query).result().to_dataframe()
print("Процент отвеченных вопросов: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Количество вопросов:", len(first_result))
first_result.head()